In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris 
import seaborn as sns
from utils import variables as var
from sklearn.linear_model import LinearRegression
#Financiero
import yfinance as yf
import quantstats as qs
import ta

In [95]:
#Using an 8Gbyte archive, we generate the main DataSets and the we export to csv
stocks=pd.read_csv(r"C:\Users\Adrian\Documents\DATA_SCIENCE_BOOTCAMP\DATA\sp500_stocks.csv")
buffet=stocks[stocks['Symbol']=='BRK-B']
waste=stocks[stocks['Symbol']=='WM']
# We mainly uses the csv archives + yfinance as an auxiliary module
gold=pd.read_csv('data/Gold_data.csv')
index_sp500=pd.read_csv('data/sp500_index.csv')
company=pd.read_csv('data/sp500_companies.csv')
buffet.to_csv('data/buffet.csv')
waste.to_csv('data/wastemanagement.csv')
waste.head()


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
1741432,2010-01-04,WM,23.382090,34.160000,34.259998,33.869999,34.009998,2058800.0
1741433,2010-01-05,WM,23.279421,34.009998,34.220001,33.779999,34.220001,2961700.0
1741434,2010-01-06,WM,23.272572,34.000000,34.150002,33.779999,33.869999,1636700.0
1741435,2010-01-07,WM,23.327345,34.080002,34.119999,33.820000,33.939999,1406700.0
1741436,2010-01-08,WM,23.436859,34.240002,34.330002,33.669998,33.919998,1451900.0


In [96]:
#Para hacer el índice de magnificas, vamos a generar los 5 datasets de cada acción y luego lo concatenaremos más adelante

msft=stocks[stocks['Symbol']=='MSFT'].set_index('Date')
appl=stocks[stocks['Symbol']=='AAPL'].set_index('Date')
meta=stocks[stocks['Symbol']=='META'].set_index('Date')
goog=stocks[stocks['Symbol']=='GOOG'].set_index('Date')
amzn=stocks[stocks['Symbol']=='AMZN'].set_index('Date')
amzn.head()

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2010-01-04,AMZN,6.6950,6.6950,6.8305,6.6570,6.8125,151998000.0
2010-01-05,AMZN,6.7345,6.7345,6.7740,6.5905,6.6715,177038000.0
2010-01-06,AMZN,6.6125,6.6125,6.7365,6.5825,6.7300,143576000.0
2010-01-07,AMZN,6.5000,6.5000,6.6160,6.4400,6.6005,220604000.0
2010-01-08,AMZN,6.6760,6.6760,6.6840,6.4515,6.5280,196610000.0


## ESTADO Y LIMPIEZA DE LOS DATOS

In [97]:
#VAMOS A MIRAR EL ESTADO DE LOS DATOS DE CADA COLUMNA
gold.info()
print('//////////////////////////////////////////////////// \n')
index_sp500.info()
print('//////////////////////////////////////////////////// \n')
company.info()
print('//////////////////////////////////////////////////// \n')
buffet.info()
print('//////////////////////////////////////////////////// \n')
waste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    5929 non-null   object 
 1   open    5929 non-null   float64
 2   high    5929 non-null   float64
 3   low     5929 non-null   float64
 4   close   5929 non-null   float64
 5   volume  5929 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 278.1+ KB
//////////////////////////////////////////////////// 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2518 non-null   object 
 1   S&P500  2518 non-null   float64
dtypes: float64(1), object(1)
memory usage: 39.5+ KB
//////////////////////////////////////////////////// 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column 

- PARA NUESTRO ESTUDIO NO NECESITAMOS PRECIO APERTURA, MAXIMO DIARIO Y MÍNIMO DIARIO: SE DROPEARAN ESAS COLUMNAS
- NECESITAMOS GENERAR UNA COLUMNA CON LA FECHA EN FORMATO DATETIME YA QUE NOS VIENE COMO OBJECT
- EN EL DATASET COMPANIES LOS VALORES NULOS SON CASI INAPRECIABLES COMO PARA DESVIAR EL ESTUDIO

#### OJO 
LOS VALORES NULOS QUE TENEMOS HACEN REFERENCIA A LOS DÍAS FESTIVOS, CUYO VOLUMEN DE NEGOCIACION ES 0 Y EL PRECIO NO VARÍA EN NINGUN MOMENTO, CON LO QUE HAREMOS UN DROP DE ESAS FILAS

### ELIMINADO VALORES FESTIVOS

In [98]:
company=company.dropna()
company.info()
buffet = buffet.dropna()
waste=waste.dropna()
waste.info()
msft=msft.dropna()
appl=appl.dropna()
meta=meta.dropna()
goog=goog.dropna()
amzn=amzn.dropna()
amzn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 449 entries, 0 to 502
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exchange             449 non-null    object 
 1   Symbol               449 non-null    object 
 2   Shortname            449 non-null    object 
 3   Longname             449 non-null    object 
 4   Sector               449 non-null    object 
 5   Industry             449 non-null    object 
 6   Currentprice         449 non-null    float64
 7   Marketcap            449 non-null    int64  
 8   Ebitda               449 non-null    float64
 9   Revenuegrowth        449 non-null    float64
 10  City                 449 non-null    object 
 11  State                449 non-null    object 
 12  Country              449 non-null    object 
 13  Fulltimeemployees    449 non-null    float64
 14  Longbusinesssummary  449 non-null    object 
 15  Weight               449 non-null    float64


### ELIMINADO VALORES DE TRADING

In [99]:
lista_columnas=['open','high','low']
lista_columnas1=['Open','High','Low','Adj Close']


gold=gold.drop(lista_columnas, axis=1)
gold.info()
buffet=buffet.drop(lista_columnas1,axis=1)
waste=waste.drop(lista_columnas1,axis=1)
msft=msft.drop(lista_columnas1,axis=1)
appl=appl.drop(lista_columnas1,axis=1)
meta=meta.drop(lista_columnas1,axis=1)
goog=goog.drop(lista_columnas1,axis=1)
amzn=amzn.drop(lista_columnas1,axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    5929 non-null   object 
 1   close   5929 non-null   float64
 2   volume  5929 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 139.1+ KB


### CONCATENACION MAGNIFICAS

In [100]:
magnificas=pd.concat([msft,appl,meta,goog,amzn],axis=1,join='inner')

magnificas.head()

,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume
Date,,,,,,,,,,,,,,,
2012-05-18,MSFT,29.270000,56205300.0,AAPL,18.942142,732292400.0,META,38.230000,573576400.0,GOOG,14.953949,239835606.0,AMZN,10.6925,104634000.0
2012-05-21,MSFT,29.750000,38787900.0,AAPL,20.045713,631106000.0,META,34.029999,168192700.0,GOOG,15.295419,123477094.0,AMZN,10.9055,71596000.0
2012-05-22,MSFT,29.760000,39504900.0,AAPL,19.891787,694870400.0,META,31.000000,101786600.0,GOOG,14.963912,122533571.0,AMZN,10.7665,74662000.0
2012-05-23,MSFT,29.110001,65171000.0,AAPL,20.377144,584897600.0,META,32.000000,73600000.0,GOOG,15.179603,127600492.0,AMZN,10.8640,84876000.0
2012-05-24,MSFT,29.070000,52575000.0,AAPL,20.190001,496230000.0,META,33.029999,50237200.0,GOOG,15.035145,75935562.0,AMZN,10.7620,62822000.0


### CREACION COLUMNAS DATE TIME Y AÑO

In [101]:
#A PARTIR DEL DATE OBJECT LO PASAMOS A DATETIME Y GENERAMOS LA COLUMNA DATETIME
gold['DateTime']=pd.to_datetime(gold['date'])
#CON EL DATE TIME GENERAMOS LA COLUMNA YEAR
gold['year']=gold['DateTime'].dt.year

gold.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      5929 non-null   object        
 1   close     5929 non-null   float64       
 2   volume    5929 non-null   int64         
 3   DateTime  5929 non-null   datetime64[ns]
 4   year      5929 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(1)
memory usage: 208.6+ KB


In [102]:
index_sp500['DateTime']=pd.to_datetime(index_sp500['Date'])
index_sp500['year']=index_sp500['DateTime'].dt.year
index_sp500.info()

buffet['DateTime']=pd.to_datetime(buffet['Date'])
index_sp500['year']=index_sp500['DateTime'].dt.year

magnificas=magnificas.reset_index()
magnificas['DateTime']=pd.to_datetime(magnificas['Date'])
magnificas['year']=magnificas['DateTime'].dt.year

waste['DateTime']=pd.to_datetime(waste['Date'])
waste['year']=waste['DateTime'].dt.year
magnificas.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2518 non-null   object        
 1   S&P500    2518 non-null   float64       
 2   DateTime  2518 non-null   datetime64[ns]
 3   year      2518 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 69.0+ KB


,Date,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume,Symbol,Close,Volume,DateTime,year
0,2012-05-18,MSFT,29.270000,56205300.0,AAPL,18.942142,732292400.0,META,38.230000,573576400.0,GOOG,14.953949,239835606.0,AMZN,10.6925,104634000.0,2012-05-18,2012
1,2012-05-21,MSFT,29.750000,38787900.0,AAPL,20.045713,631106000.0,META,34.029999,168192700.0,GOOG,15.295419,123477094.0,AMZN,10.9055,71596000.0,2012-05-21,2012
2,2012-05-22,MSFT,29.760000,39504900.0,AAPL,19.891787,694870400.0,META,31.000000,101786600.0,GOOG,14.963912,122533571.0,AMZN,10.7665,74662000.0,2012-05-22,2012
3,2012-05-23,MSFT,29.110001,65171000.0,AAPL,20.377144,584897600.0,META,32.000000,73600000.0,GOOG,15.179603,127600492.0,AMZN,10.8640,84876000.0,2012-05-23,2012
4,2012-05-24,MSFT,29.070000,52575000.0,AAPL,20.190001,496230000.0,META,33.029999,50237200.0,GOOG,15.035145,75935562.0,AMZN,10.7620,62822000.0,2012-05-24,2012


### CREACION INDICE MAGNIFICAS

In [103]:
#Para ello sumaremos los valores al cierre y el volumen de negociación

magnificas['tkr'] = magnificas.iloc[:, 1] + magnificas.iloc[:, 4] + magnificas.iloc[:, 7] + magnificas.iloc[:, 10] + magnificas.iloc[:, 13]
magnificas['Valor_indice'] = magnificas.iloc[:, 2] + magnificas.iloc[:, 5] + magnificas.iloc[:, 8] + magnificas.iloc[:, 11] + magnificas.iloc[:, 14]
magnificas['Volumen_total'] = magnificas.iloc[:, 3] + magnificas.iloc[:, 6] + magnificas.iloc[:, 9] + magnificas.iloc[:, 12] + magnificas.iloc[:, 15]
magnificas = magnificas.drop(magnificas.columns[0:15], axis=1)
magnificas

#Finalmente guardamos en índice en un csv
magnificas.to_csv('data/magnificas.csv')

## ANÁLISIS PRIMERA Y SEGUNDA HIPÓTESIS:

### EL SP500 ES EL MEJOR INDICE PARA INVERTIR SEGUIDO DEL ORO

In [104]:
# En primer lugar vamos a obtener los principales estadisticos de cada uno de los indices

gold[['close','volume']].describe()


,close,volume
count,5929.000000,5929.000000
mean,1120.855726,4303.936246
std,539.295822,24451.870855
min,255.100006,0.000000
25%,621.500000,20.000000
50%,1224.500000,105.000000
75%,1564.599976,397.000000
max,2390.800049,386334.000000


In [105]:
index_sp500[['S&P500']].describe()

,S&P500
count,2518.000000
mean,3100.992514
std,941.271669
min,1829.080000
25%,2175.452500
50%,2853.555000
75%,3986.317500
max,5254.350000
